In [24]:
from google.cloud import storage
import gcsfs
import pickle
import os
from typing import Optional
import pandas as pd
import chromadb
from chromadb.config import Settings
import getpass
from langchain_google_genai import GoogleGenerativeAIEmbeddings

TypeError: cannot pickle 'classmethod' object

In [21]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "myraglangchainapp-fb8241de750a.json"

In [10]:
def content_type_finder(file_name):
    if file_name:
        if file_name.endswith('.csv'):
            content_type = 'text/csv'
            folder_name = 'csvfiles'
        elif file_name.endswith('.txt'):
            content_type = 'text/plain'
            folder_name = 'txtfiles'
        elif file_name.endswith('.json'):
            content_type = 'application/json'
            folder_name = 'jsonfiles'
        else:
            content_type = 'application/octet-stream'
            folder_name = 'others'
    elif text_input:
        if urllib.parse.urlparse(text_input).scheme in ('http', 'https'):
            content_type = 'text/plain'
            folder_name = 'urls'
        else:
            content_type = 'text/plain'
            folder_name = 'plaininputtexts'
    else:
        content_type = 'application/octet-stream'
        folder_name = 'others'

    return content_type, folder_name

In [12]:
def upload_to_gcp_bucket(file_path, bucket_name):
    # Get the file name and its extension
    file_name = os.path.basename(file_path)
    content_type, folder_name = content_type_finder(file_name)
    
    # Initialize GCP client
    client = storage.Client()
    bucket = client.get_bucket(bucket_name)
    
    # Create a blob object
    blob = bucket.blob(f"{folder_name}/{file_name}")
    
    # Upload file to the bucket with appropriate content type
    with open(file_path, 'rb') as file_data:
        blob.upload_from_file(file_data, content_type=content_type)
    
    print(f"File '{file_name}' uploaded to '{bucket_name}', / {folder_name} with content type '{content_type}'")

# Example usage
#file_path = 'sample1.txt'  # Provide the correct file path
file_path = 'color_web_safe - color_web_safe.csv'
bucket_name = 'userdatafiles'
upload_to_gcp_bucket(file_path, bucket_name)


File 'color_web_safe - color_web_safe.csv' uploaded to 'userdatafiles', / csvfiles with content type 'text/csv'


In [15]:
def download_text_file(bucket_name, file_name):
    """Download a text file from GCS bucket and return its content."""
    client = storage.Client()
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(file_name)

    # Download the content of the file
    content = blob.download_as_text()
    return content

In [ ]:
def store_in_chromadb(content, collection_name):
    """Store content in ChromaDB."""
    # Initialize ChromaDB
    client = chromadb.Client(Settings(chroma_db_impl="duckdb+parquet", persist_directory="chroma_db"))

    # Create a new collection or get existing
    collection = client.create_collection(name=collection_name)

    # Add the content to the collection
    collection.add(documents=[content], metadatas=[{"source": "gcs", "file_name": file_name}])

    print(f"Content stored in ChromaDB collection '{collection_name}'.")
